<a href="https://colab.research.google.com/github/Ayikanying-ux/keras_core_api/blob/main/sequential_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### The Sequential model

In [1]:
import tensorflow as tf
import keras
from keras import layers

### When to use a Sequential model
* A ```Sequential``` model is appropriate for a plain stack of layers where each layer has exactly on input tensor and one output tensor

In [2]:
model = keras.Sequential(
    [
        layers.Dense(2, activation='relu', name='layer1'),
        layers.Dense(3, activation='relu', name='layer2'),
        layers.Dense(4, name='layer3'),
    ]
)

x=tf.ones((3, 3))
y = model(x)

In [3]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))

A sequential model is not appropriate when:
* Your model has multiple inputs or multiple outputs
* Any of your layers has multiple inputs of multiple outputs
* You need to do layer sharing
* You want non-linear topology(e.g a residual connection, a multi-branch model)

### Creating a Sequential model
You can create a Sequential model by passing a list of layers ot the Sequential constructor:

In [4]:
model = keras.Sequential(
    [
        layers.Dense(2, activation='relu'),
        layers.Dense(3, activation='relu'),
        layers.Dense(4),
    ]
)



---
* It's layers are accessible via the ```layers``` attribute


In [5]:
model.layers



---
* You can also create a Sequential model incrementally via the ```add()``` method


In [6]:
model = keras.Sequential()
model.add(layers.Dense(2, activation='relu'))
model.add(layers.Dense(3, activation='relu'))
model.add(layers.Dense(4))

* Note that there's also a corresponding pop() method to remove layers: a Sequential model behaves very much like a list of layers.

In [7]:
model.pop()
print(len(model.layers))  # 2

2


* Also note that the Sequential constructor accepts a name argument, just like any layer or model in Keras. This is useful to annotate TensorBoard graphs with semantically meaningful names.

In [8]:
model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(2, activation="relu", name="layer1"))
model.add(layers.Dense(3, activation="relu", name="layer2"))
model.add(layers.Dense(4, name="layer3"))

### Specifying the input shape in advance
* Generally, all layers in Keras need to know the shape of their inputs in order to be able to create their weights. So when you create a layer like this, initially, it has no weights.

In [9]:
layer = layers.Dense(3)
layer.weights

[]



---
* It creates its weights the first time it is called on an input, since the shape of the weights depends on the shape of the inputs:

In [10]:
# Call layer on a test input
x = tf.ones((1, 4))
y = layer(x)
layer.weights  # Now it has weights, of shape (4, 3) and (3,)

[<tf.Variable 'dense_6/kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[-0.22815353, -0.39108938,  0.67584586],
        [ 0.50154996, -0.07992089,  0.107324  ],
        [ 0.03392774, -0.2876765 ,  0.53579485],
        [ 0.51196945, -0.02100182, -0.19639188]], dtype=float32)>,
 <tf.Variable 'dense_6/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]



---
* Naturally, this also applies to Sequential models. When you instantiate a Sequential model without an input shape, it isn't "built": it has no weights (and calling model.weights results in an error stating just this). The weights are created when the model first sees some input data


In [11]:
model = keras.Sequential(
    [
        layers.Dense(2, activation='relu'),
        layers.Dense(3, activation='relu'),
        layers.Dense(4)
    ]
) # No weights at this stage!
# At this point, you can't do this:
# model.weights

# You also can't do this:
# model.summary()

# Call the model on a test input
x = tf.ones((1, 4))
y = model(x)
print("Number fo weights after calling the model:", len(model.weights))

Number fo weights after calling the model: 6


* Once a model is "built", you can call its summary() method to display its contents:

In [12]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (1, 2)                    10        
                                                                 
 dense_8 (Dense)             (1, 3)                    9         
                                                                 
 dense_9 (Dense)             (1, 4)                    16        
                                                                 
Total params: 35 (140.00 Byte)
Trainable params: 35 (140.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________




---
* However, it can be very useful when building a Sequential model incrementally to be able to display the summary of the model so far, including the current output shape. In this case, you should start your model by passing an Input object to your model, so that it knows its input shape from the start:


In [13]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation="relu"))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 2)                 10        
                                                                 
Total params: 10 (40.00 Byte)
Trainable params: 10 (40.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________




---
* Note that the Input object is not displayed as part of model.layers, since it isn't a layer

In [14]:
model.layers



---
* A simple alternative is to just pass an input_shape argument to your first layer:


In [15]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu", input_shape=(4,)))

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 2)                 10        
                                                                 
Total params: 10 (40.00 Byte)
Trainable params: 10 (40.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________




---
* Models built with a predefined input shape like this always have weights (even before seeing any data) and always have a defined output shape.
* In general, it's a recommended best practice to always specify the input shape of a Sequential model in advance if you know what it is.


### A common debugging workflow: add() + summary()
* When building a new Sequential architecture, it's useful to incrementally stact layers with ```add()``` and frequently print model summaries. For instance, this enables you to monitor how a stack of ```Conv2D``` and ```MaxPooling2D``` layers is downsampling image feature maps:

In [17]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))
model.add(layers.Conv2D(35, 5, strides=2, activation='relu'))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))


# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()


# The answer was: (40, 40, 32), so we can keep downsampling...

model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

model.summary()


# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPool2D())


# Finally, we add a classification layer.
model.add(layers.Dense(10))

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 123, 123, 35)      2660      
                                                                 
 conv2d_7 (Conv2D)           (None, 121, 121, 32)      10112     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 40, 40, 32)        0         
 g2D)                                                            
                                                                 
Total params: 12772 (49.89 KB)
Trainable params: 12772 (49.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 123, 123, 35)      2660   

### What to do once you have a model
Once your model architecture is ready, you will want to:
- Train your model, evaluate it, and run inference. See our guide to training & evaluation with the built-in loops
- Save your model to disk and restore it. See our guide to serialization & saving.
- Speed up model training by leveraging multiple GPUs. See our guide to multi-GPU and distributed training.



---
### Feature extraction with a Sequential model
Once a Sequential model has been built, it behaves like a Functional API model. This means that every layer has an input and output attribute. These attributes can be used to do neat things, like quickly creating a model that extracts the outputs of all intermediate layers in a Sequential model:

In [21]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu")
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

In [22]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name="my_intermediate_layer").output,
)
# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)